In [1]:
%matplotlib widget
#%matplotlib inline

In [2]:
# initialization
from rayoptics.environment import *
from rayoptics.optical.model_enums import DecenterType as dec

In [3]:
import pprint
import json

In [4]:
root_pth = Path(rayoptics.__file__).resolve().parent

In [5]:
opm = OpticalModel()
sm = opm.seq_model
osp = opm.optical_spec
pm = opm.parax_model
em = opm.ele_model

### Define first order aperture and field for model

In [6]:
osp.pupil = PupilSpec(osp, key=['object', 'pupil'], value=1.)
osp.field_of_view = FieldSpec(osp, key=['object', 'angle'], flds=[0.])
osp.spectral_region = WvlSpec([(555.0, 1.0)], ref_wl=0)

### Enter sequential model

In [7]:
opm.radius_mode = True

sm.gaps[0].thi=1e10

opm.add_dummy_plane(t=5.0)
sm.set_stop()

opm.add_mirror(t=-5.0)
sm.ifcs[2].decenter = srf.DecenterData(dec.BEND, alpha=60.)

opm.add_mirror(t=5.0)
sm.ifcs[3].decenter = srf.DecenterData(dec.BEND, alpha=-60.)

opm.add_mirror(t=-5.0)
sm.ifcs[4].decenter = srf.DecenterData(dec.BEND, alpha=-45.)

In [8]:
opm.update_model()

In [9]:
sm.list_model()
print('')
sm.list_decenters()

             r            t        medium     mode   zdr      sd
 Obj:     0.000000  1.00000e+10       air   transmit  1      0.0000
Stop:     0.000000      5.00000       air   transmit  1     0.50000
   2:     0.000000     -5.00000       air    reflect -1      1.0000
   3:     0.000000      5.00000       air    reflect  1      1.0000
   4:     0.000000     -5.00000       air    reflect -1     0.70711
 Img:    -0.000000      0.00000             transmit -1     0.50000

             thi    medium/mode    type        y       alpha
   0:  1.00000e+10          air
   1:      5.00000          air
   2:                   reflect    BEND     0.0000     60.000
   2:     -5.00000          air
   3:                   reflect    BEND     0.0000    -60.000
   3:      5.00000          air
   4:                   reflect    BEND     0.0000    -45.000
   4:     -5.00000          air
   5:                  transmit


In [10]:
opm.specsheet.imager

IdealImager(m=None, s=-inf, sp=None, tt=None, f=None)

In [11]:
layout_plt = plt.figure(FigureClass=InteractiveLayout, opt_model=opm,
                        do_draw_rays=True, do_paraxial_layout=False,
                        refresh_gui=None).plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
em.list_elements()

0: DummyInterface (DummyInterface): Surface(profile=Spherical(c=0.0), interact_mode=transmit)
1: AirGap (AirGap): Gap(t=5.0, medium=Air)
2: Mirror (Mirror): Mirror: Spherical(c=0.0), t=0.0500, sd=1.0000
3: AirGap (AirGap): Gap(t=-5.0, medium=Air)
4: Mirror (Mirror): Mirror: Spherical(c=0.0), t=0.0500, sd=1.0000
5: AirGap (AirGap): Gap(t=5.0, medium=Air)
6: Mirror (Mirror): Mirror: Spherical(c=0.0), t=0.0354, sd=0.7071
7: AirGap (AirGap): Gap(t=-5.0, medium=Air)


In [13]:
for i,e in enumerate(em.elements):
    if hasattr(e, 'get_thi'):
        print(i, e.get_thi(), e.z_dir, sm.z_dir[e.reference_idx()], e.reference_interface().normal(np.array([0., 0., 0.])))

2 0.05000000002933008 -1.0 -1.0 [-0. -0.  1.]
4 0.050000000054330085 1.0 1.0 [-0. -0.  1.]
6 0.035355339114128186 -1.0 -1.0 [-0. -0.  1.]


In [14]:
for i,trfm in enumerate(sm.gbl_tfrms):
    print("{}: {:12.7g} {:12.7g} {:12.7g}".format(i, trfm[1][0], trfm[1][1], trfm[1][2]))

0:            0            0       -1e+10
1:            0            0            0
2:            0            0            5
3:            0    -4.330127          7.5
4:            0    -4.330127         12.5
5:            0     0.669873         12.5


In [15]:
pt0 = np.array([0., 0., 0.])
dir0 = np.array([0., 0., 1.])
wvl = sm.central_wavelength()
oar = rt.trace(sm, pt0, dir0, wvl)
list_ray(oar[0])

            X            Y            Z           L            M            N               Len
  0:      0.00000      0.00000            0     0.000000     0.000000     1.000000        1e+10
  1:      0.00000      0.00000            0     0.000000     0.000000     1.000000            5
  2:      0.00000      0.00000            0     0.000000    -0.866025    -0.500000            5
  3:      0.00000      0.00000            0     0.000000    -0.866025     0.500000            5
  4:      0.00000      0.00000            0     0.000000     0.707107    -0.707107            5
  5:      0.00000      0.00000            0     0.000000     0.000000    -1.000000            0


In [16]:
tfrms = sm.gbl_tfrms
print("            X            Y            Z        z_dir     SRL"
      "          SRM          SRN")
for i, r in enumerate(oar[mc.ray][0:]):
    rot, trns = tfrms[i]
    p = rot.dot(r[mc.p]) + trns
    snr = rot.dot(r[mc.nrml])
    print("{:3d}: {:12.5f} {:12.5f} {:12.5g}   {:4.0f} {:12.6f} {:12.6f} {:12.6f}"
          .format(i, p[0], p[1], p[2], sm.z_dir[i], snr[0], snr[1], snr[2]))

            X            Y            Z        z_dir     SRL          SRM          SRN
  0:      0.00000      0.00000       -1e+10      1     0.000000     0.000000     1.000000
  1:      0.00000      0.00000            0      1     0.000000     0.000000     1.000000
  2:      0.00000      0.00000            5     -1     0.000000     0.866025     0.500000
  3:      0.00000     -4.33013          7.5      1     0.000000     0.866025     0.500000
  4:      0.00000     -4.33013         12.5     -1     0.000000    -0.707107     0.707107
  5:      0.00000      0.66987         12.5     -1     0.000000    -1.000000     0.000000


In [17]:
list_ray(oar,tfrms=sm.gbl_tfrms)

            X            Y            Z           L            M            N               Len
  0:      0.00000      0.00000       -1e+10     0.000000     0.000000     1.000000        1e+10
  1:      0.00000      0.00000            0     0.000000     0.000000     1.000000            5
  2:      0.00000      0.00000            5     0.000000    -0.866025     0.500000            5
  3:      0.00000     -4.33013          7.5     0.000000     0.000000     1.000000            5
  4:      0.00000     -4.33013         12.5     0.000000     1.000000     0.000000            5
  5:      0.00000      0.66987         12.5     0.000000     1.000000    -0.000000            0


In [18]:
sm.z_dir

[1.0, 1.0, -1.0, 1.0, -1.0, -1.0]

In [19]:
sm.list_decenters()

             thi    medium/mode    type        y       alpha
   0:  1.00000e+10          air
   1:      5.00000          air
   2:                   reflect    BEND     0.0000     60.000
   2:     -5.00000          air
   3:                   reflect    BEND     0.0000    -60.000
   3:      5.00000          air
   4:                   reflect    BEND     0.0000    -45.000
   4:     -5.00000          air
   5:                  transmit


In [20]:
opm.save_model('Tilted mirrors')

In [21]:
opm = open_model(root_pth/"codev/tests/folded_lenses.seq")
sm = opm.seq_model
osp = opm.optical_spec
pm = opm.parax_model
em = opm.ele_model

In [22]:
sm.list_model()
print('')
sm.list_decenters()

             c            t        medium     mode   zdr      sd
 Obj:     0.000000  1.00000e+11       air   transmit  1      0.0000
Stop:     0.000000      5.00000       air   transmit  1     0.50000
   2:     0.000000     -2.25000       air    reflect -1      1.0000
   3:    -0.670000    -0.250000     N-BK7   transmit -1     0.50000
   4:     0.000000   -0.0500000       air   transmit -1     0.48040
   5:     0.000000    -0.250000     N-BK7   transmit -1     0.47123
   6:    -0.770950     -2.25000       air   transmit -1     0.43242
   7:     0.000000      2.25000       air    reflect  1     0.86202
   8:    -0.670000     0.250000     N-BK7   transmit  1     0.42812
   9:     0.000000    0.0500000       air   transmit  1     0.45957
  10:     0.000000     0.250000     N-BK7   transmit  1     0.46726
  11:    -0.592250      2.25000       air   transmit  1     0.48523
  12:     0.000000     -5.00000       air    reflect -1     0.68285
 Img:     0.000000      0.00000             transmi

In [23]:
layout_plt = plt.figure(FigureClass=InteractiveLayout, opt_model=opm,
                        do_draw_rays=True, do_paraxial_layout=False,
                        refresh_gui=None).plot()
#layout_plt.update_data()
#layout_plt.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
layout_plt.view_bbox

array([[-0.90409953, -4.88041884],
       [13.08199055,  0.62657171]])

In [25]:
opm.specsheet.imager

IdealImager(m=-0.0, s=-inf, sp=2564.3352539285456, tt=inf, f=2564.3352539285456)

In [26]:
ss = opm.specsheet
print(ss)

infinite conjugates:
imager: IdealImager(m=-0.0, s=-inf, sp=2564.3352539285456, tt=inf, f=2564.3352539285456)
imager inputs: {'s': -inf}
frozen imager inputs: [True, True, True, True, False]
etendue inputs:
  field:    {'object': {'angle': 0.0}, 'image': {}}
  aperture: {'object': {'pupil': 1.0}, 'image': {}}
etendue values:
  field:    {'object': {'angle': 0.0}, 'image': {'height': 0.0}}
  aperture:
    object: {'pupil': 1.0, 'NA': 0, 'f/#': 0}
    image:  {'f/#': -2564.3352539285456, 'NA': 0.00019498230183806055, 'pupil': -1.0}


In [27]:
layout_plt.ele_bbox

array([[ 0.        , -4.88041884],
       [13.08199055,  0.62657171]])

In [28]:
pm.first_order_data()

efl                2564
ffl               -2600
pp1              -35.67
bfl                2615
ppk              -51.08
f/#                2564
m                0.9781
red            -3.9e+07
obj_dist          1e+11
obj_ang               1
enp_dist             -0
enp_radius          0.5
na obj            5e-12
n obj                 1
img_dist             -5
img_ht            44.76
exp_dist           15.9
exp_radius       0.4931
na img         0.000195
n img                -1
optical invariant     0.008728


In [29]:
print(json.dumps(ss.imager_inputs))

{"s": -Infinity}


In [30]:
pprint.pprint(ss.etendue_values['field'], indent=4)

{'image': {'height': 0.0}, 'object': {'angle': 0.0}}


In [31]:
ss.conjugate_type

'infinite'

In [32]:
ss.partition_defined()

(None, 1)

In [33]:
for i,s in enumerate(sm.ifcs):
    print(i, sm.z_dir[i], s.profile_cv)

0 1.0 0.0
1 1.0 0.0
2 -1.0 0.0
3 -1.0 -0.67
4 -1.0 0.0
5 -1.0 0.0
6 -1.0 -0.77095
7 1.0 0.0
8 1.0 -0.67
9 1.0 0.0
10 1.0 0.0
11 1.0 -0.59225
12 -1.0 0.0
13 -1.0 0.0


In [34]:
em.list_elements()

0: Aperture Stop (DummyInterface): Surface(profile=Spherical(c=0.0), interact_mode=transmit)
1: AirGap Aperture Stop-M1 (AirGap): Gap(t=5.0, medium=Air)
2: M1 (Mirror): Mirror: Spherical(c=0.0), t=0.0500, sd=1.0000
3: AirGap M1-E1 (AirGap): Gap(t=-2.25, medium=Air)
4: E1 (Element): Element: Spherical(c=-0.67), Spherical(c=0.0), t=-0.2500, sd=0.5000, glass: N-BK7
5: AirGap E1-E2 (AirGap): Gap(t=-0.05, medium=Air)
6: E2 (Element): Element: Spherical(c=0.0), Spherical(c=-0.77095), t=-0.2500, sd=0.4712, glass: N-BK7
7: AirGap E2-M3 (AirGap): Gap(t=-2.25, medium=Air)
8: M3 (Mirror): Mirror: Spherical(c=0.0), t=0.0431, sd=0.8620
9: AirGap M3-E3 (AirGap): Gap(t=2.25, medium=Air)
10: E3 (Element): Element: Spherical(c=-0.67), Spherical(c=0.0), t=0.2500, sd=0.4596, glass: N-BK7
11: AirGap E3-E4 (AirGap): Gap(t=0.05, medium=Air)
12: E4 (Element): Element: Spherical(c=0.0), Spherical(c=-0.59225), t=0.2500, sd=0.4852, glass: N-BK7
13: AirGap E4-M5 (AirGap): Gap(t=2.25, medium=Air)
14: M5 (Mirror):

In [35]:
for i,e in enumerate(em.elements):
    if hasattr(e, 'get_thi'):
        thi = e.get_thi()
        z_dir = sm.z_dir[e.reference_idx()]
        normal_00 = e.reference_interface().normal(np.array([0., 0., 0.]))
        print(i, thi, -z_dir, normal_00[2])

2 0.05000000000293301 1.0 1.0
8 0.043101242814776786 -1.0 1.0
14 0.034142519629751976 1.0 1.0


In [36]:
sm.list_model()

             c            t        medium     mode   zdr      sd
 Obj:     0.000000  1.00000e+11       air   transmit  1      0.0000
Stop:     0.000000      5.00000       air   transmit  1     0.50000
   2:     0.000000     -2.25000       air    reflect -1      1.0000
   3:    -0.670000    -0.250000     N-BK7   transmit -1     0.50000
   4:     0.000000   -0.0500000       air   transmit -1     0.48040
   5:     0.000000    -0.250000     N-BK7   transmit -1     0.47123
   6:    -0.770950     -2.25000       air   transmit -1     0.43242
   7:     0.000000      2.25000       air    reflect  1     0.86202
   8:    -0.670000     0.250000     N-BK7   transmit  1     0.42812
   9:     0.000000    0.0500000       air   transmit  1     0.45957
  10:     0.000000     0.250000     N-BK7   transmit  1     0.46726
  11:    -0.592250      2.25000       air   transmit  1     0.48523
  12:     0.000000     -5.00000       air    reflect -1     0.68285
 Img:     0.000000      0.00000             transmi

In [37]:
m1 = em.elements[2]
str(m1)

'Mirror: Spherical(c=0.0), t=0.0500, sd=1.0000'

In [38]:
m1.tfrm

(array([[ 1.       ,  0.       ,  0.       ],
        [ 0.       ,  0.5      ,  0.8660254],
        [ 0.       , -0.8660254,  0.5      ]]),
 array([0., 0., 5.]))

In [39]:
m2 = em.elements[3]

In [40]:
str(m2)

'Gap(t=-2.25, medium=Air)'

In [41]:
m2.tfrm

(array([[ 1.       ,  0.       ,  0.       ],
        [ 0.       ,  0.5      ,  0.8660254],
        [ 0.       , -0.8660254,  0.5      ]]),
 array([0., 0., 5.]))

In [42]:
for i,e in enumerate(em.elements):
    ri = e.reference_interface()
    rifc_indx = sm.ifcs.index(ri)
    print(i, e.label, ri.interact_mode, e.tfrm[0].T[2], e.reference_interface().profile.normal(np.array([0., 0., 0.])))

0 Aperture Stop transmit [0. 0. 1.] [-0. -0.  1.]
1 AirGap Aperture Stop-M1 transmit [0. 0. 1.] [-0. -0.  1.]
2 M1 reflect [0.        0.8660254 0.5      ] [-0. -0.  1.]
3 AirGap M1-E1 reflect [0.        0.8660254 0.5      ] [-0. -0.  1.]
4 E1 transmit [ 0.         0.8660254 -0.5      ] [0. 0. 1.]
5 AirGap E1-E2 transmit [ 0.         0.8660254 -0.5      ] [-0. -0.  1.]
6 E2 transmit [ 0.         0.8660254 -0.5      ] [-0. -0.  1.]
7 AirGap E2-M3 transmit [ 0.         0.8660254 -0.5      ] [0. 0. 1.]
8 M3 reflect [0.        0.8660254 0.5      ] [-0. -0.  1.]
9 AirGap M3-E3 reflect [0.        0.8660254 0.5      ] [-0. -0.  1.]
10 E3 transmit [ 0.00000000e+00 -1.48741681e-17  1.00000000e+00] [0. 0. 1.]
11 AirGap E3-E4 transmit [ 0.00000000e+00 -1.48741681e-17  1.00000000e+00] [-0. -0.  1.]
12 E4 transmit [ 0.00000000e+00 -1.48741681e-17  1.00000000e+00] [-0. -0.  1.]
13 AirGap E4-M5 transmit [ 0.00000000e+00 -1.48741681e-17  1.00000000e+00] [0. 0. 1.]
14 M5 reflect [ 0.         -0.70710678

In [43]:
for i,e in enumerate(em.elements):
    ri = e.reference_interface()
    rifc_indx = sm.ifcs.index(ri)
    print(i, e.label, ri.interact_mode, sm.z_dir[rifc_indx])

0 Aperture Stop transmit 1.0
1 AirGap Aperture Stop-M1 transmit 1.0
2 M1 reflect -1.0
3 AirGap M1-E1 reflect -1.0
4 E1 transmit -1.0
5 AirGap E1-E2 transmit -1.0
6 E2 transmit -1.0
7 AirGap E2-M3 transmit -1.0
8 M3 reflect 1.0
9 AirGap M3-E3 reflect 1.0
10 E3 transmit 1.0
11 AirGap E3-E4 transmit 1.0
12 E4 transmit 1.0
13 AirGap E4-M5 transmit 1.0
14 M5 reflect -1.0
15 AirGap M5-Image reflect -1.0
16 Image transmit -1.0


In [44]:
sm.list_model()

             c            t        medium     mode   zdr      sd
 Obj:     0.000000  1.00000e+11       air   transmit  1      0.0000
Stop:     0.000000      5.00000       air   transmit  1     0.50000
   2:     0.000000     -2.25000       air    reflect -1      1.0000
   3:    -0.670000    -0.250000     N-BK7   transmit -1     0.50000
   4:     0.000000   -0.0500000       air   transmit -1     0.48040
   5:     0.000000    -0.250000     N-BK7   transmit -1     0.47123
   6:    -0.770950     -2.25000       air   transmit -1     0.43242
   7:     0.000000      2.25000       air    reflect  1     0.86202
   8:    -0.670000     0.250000     N-BK7   transmit  1     0.42812
   9:     0.000000    0.0500000       air   transmit  1     0.45957
  10:     0.000000     0.250000     N-BK7   transmit  1     0.46726
  11:    -0.592250      2.25000       air   transmit  1     0.48523
  12:     0.000000     -5.00000       air    reflect -1     0.68285
 Img:     0.000000      0.00000             transmi

In [45]:
m2.tfrm[0][2]

array([ 0.       , -0.8660254,  0.5      ])

In [46]:
d = m2.tfrm[0].dot(np.array([0., 0., 1.])); d

array([0.       , 0.8660254, 0.5      ])

In [47]:
m2.tfrm[0].T[2]

array([0.       , 0.8660254, 0.5      ])

In [48]:
m2.tfrm[0].transpose()[2]

array([0.       , 0.8660254, 0.5      ])

In [49]:
m2_indx = sm.ifcs.index(m2.reference_interface()); m2_indx

2

In [50]:
sm.gbl_tfrms[m2_indx][0].T

array([[ 1.       ,  0.       ,  0.       ],
       [ 0.       ,  0.5      , -0.8660254],
       [ 0.       ,  0.8660254,  0.5      ]])

In [51]:
m2.reference_interface().profile.normal(np.array([0., 0., 0.]))

array([-0., -0.,  1.])

In [52]:
sm.z_dir

[1.0, 1.0, -1.0, -1.0, -1.0, -1.0, -1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -1.0, -1.0]

In [53]:
sm.list_model()

             c            t        medium     mode   zdr      sd
 Obj:     0.000000  1.00000e+11       air   transmit  1      0.0000
Stop:     0.000000      5.00000       air   transmit  1     0.50000
   2:     0.000000     -2.25000       air    reflect -1      1.0000
   3:    -0.670000    -0.250000     N-BK7   transmit -1     0.50000
   4:     0.000000   -0.0500000       air   transmit -1     0.48040
   5:     0.000000    -0.250000     N-BK7   transmit -1     0.47123
   6:    -0.770950     -2.25000       air   transmit -1     0.43242
   7:     0.000000      2.25000       air    reflect  1     0.86202
   8:    -0.670000     0.250000     N-BK7   transmit  1     0.42812
   9:     0.000000    0.0500000       air   transmit  1     0.45957
  10:     0.000000     0.250000     N-BK7   transmit  1     0.46726
  11:    -0.592250      2.25000       air   transmit  1     0.48523
  12:     0.000000     -5.00000       air    reflect -1     0.68285
 Img:     0.000000      0.00000             transmi

In [54]:
pm.list_lens()

       ax_ray_ht    ax_ray_slp
 0:            0         5e-12
 1:          0.5         5e-12
 2:          0.5         5e-12
 3:          0.5     -0.173622
 4:      0.47141     -0.173622
 5:      0.46273     -0.173622
 6:      0.43414  -0.000154742
 7:      0.43379  -0.000154742
 8:      0.43345      0.150356
 9:       0.4582      0.150356
10:      0.46572      0.150356
11:      0.49048  -0.000194982
12:      0.49004  -0.000194982
13:      0.48906  -0.000194982

       pr_ray_ht    pr_ray_slp
 0:  -1.7455e+09     0.0174551
 1:            0     0.0174551
 2:     0.087275     0.0174551
 3:      0.12655    -0.0264883
 4:      0.12219    -0.0264883
 5:      0.12086    -0.0264883
 6:       0.1165     0.0200614
 7:      0.16164     0.0200614
 8:      0.20678     0.0918638
 9:       0.2219     0.0918638
10:       0.2265     0.0918638
11:      0.24162     0.0176979
12:      0.28144     0.0176979
13:      0.36993     0.0176979

            power           tau        index    type
 0:            

In [55]:
pupil = [0., 1.]
fi = 0
wl = osp.spectral_region.reference_wvl
fld, wvl, foc = osp.lookup_fld_wvl_focus(fi, wl)
ray, ray_op, wvl = trace_base(opm, pupil, fld, wvl)
list_ray(ray)

            X            Y            Z           L            M            N               Len
  0:      0.00000      0.00000            0     0.000000     0.000000     1.000000        1e+11
  1:      0.00000      0.50000            0     0.000000     0.000000     1.000000        4.134
  2:      0.00000      1.00000            0     0.000000    -0.866025    -0.500000       3.2023
  3:      0.00000      0.50000    -0.086242     0.000000    -0.118848    -0.992913      0.16493
  4:      0.00000      0.48040            0     0.000000    -0.180443    -0.983585     0.050834
  5:      0.00000      0.47123            0     0.000000    -0.118848    -0.992913      0.32652
  6:      0.00000      0.43242    -0.074201     0.000000    -0.000985    -1.000000       1.4293
  7:      0.00000      0.86202            0     0.000000    -0.866518     0.499147       2.9338
  8:      0.00000      0.42812    -0.062719     0.000000     0.100059     0.994981       0.3143
  9:      0.00000      0.45957          

In [56]:
def ray_lister(ray):
    colHeader = "            X            Y            Z           L" \
                "          M          N          SRL        SRM        SRN          Len"
    print(colHeader)

    colFormats = "{:3d}: {:12.5f} {:12.5f} {:12.5g} {:10.4f} {:10.4f} " \
                 "{:10.4f} {:10.4f} {:10.4f} {:10.4f} {:12.5g}"

    for i, r in enumerate(ray):
        print(colFormats.format(i,
                                r[mc.p][0], r[mc.p][1], r[mc.p][2],
                                r[mc.d][0], r[mc.d][1], r[mc.d][2],
                                r[mc.nrml][0], r[mc.nrml][1], r[mc.nrml][2],
                                r[mc.dst]))

In [57]:
ray_lister(ray)

            X            Y            Z           L          M          N          SRL        SRM        SRN          Len
  0:      0.00000      0.00000            0     0.0000     0.0000     1.0000    -0.0000    -0.0000     1.0000        1e+11
  1:      0.00000      0.50000            0     0.0000     0.0000     1.0000    -0.0000    -0.0000     1.0000        4.134
  2:      0.00000      1.00000            0     0.0000    -0.8660    -0.5000    -0.0000    -0.0000     1.0000       3.2023
  3:      0.00000      0.50000    -0.086242     0.0000    -0.1188    -0.9929     0.0000     0.3350     0.9422      0.16493
  4:      0.00000      0.48040            0     0.0000    -0.1804    -0.9836    -0.0000    -0.0000     1.0000     0.050834
  5:      0.00000      0.47123            0     0.0000    -0.1188    -0.9929    -0.0000    -0.0000     1.0000      0.32652
  6:      0.00000      0.43242    -0.074201     0.0000    -0.0010    -1.0000     0.0000     0.3334     0.9428       1.4293
  7:      0.00000

In [58]:
sm.list_model()

             c            t        medium     mode   zdr      sd
 Obj:     0.000000  1.00000e+11       air   transmit  1      0.0000
Stop:     0.000000      5.00000       air   transmit  1     0.50000
   2:     0.000000     -2.25000       air    reflect -1      1.0000
   3:    -0.670000    -0.250000     N-BK7   transmit -1     0.50000
   4:     0.000000   -0.0500000       air   transmit -1     0.48040
   5:     0.000000    -0.250000     N-BK7   transmit -1     0.47123
   6:    -0.770950     -2.25000       air   transmit -1     0.43242
   7:     0.000000      2.25000       air    reflect  1     0.86202
   8:    -0.670000     0.250000     N-BK7   transmit  1     0.42812
   9:     0.000000    0.0500000       air   transmit  1     0.45957
  10:     0.000000     0.250000     N-BK7   transmit  1     0.46726
  11:    -0.592250      2.25000       air   transmit  1     0.48523
  12:     0.000000     -5.00000       air    reflect -1     0.68285
 Img:     0.000000      0.00000             transmi

In [59]:
%connect_info

{
  "shell_port": 51193,
  "iopub_port": 51194,
  "stdin_port": 51195,
  "control_port": 51197,
  "hb_port": 51196,
  "ip": "127.0.0.1",
  "key": "d00f3282-e1c41a54a353f0d198fbec58",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": ""
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-94d74742-1293-49d2-8129-8464c601b5f4.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.
